1.  **Free Kaggle Notebook V1 - 29 March 2024**
1.  **Please read instructions carefully**
1. **Select Accelerator GPU T4 x2 and set Internet On**
1. **Most updated version with config of this notebook will be on >**
1. **https://www.patreon.com/posts/97769887**
1.  **Tutorial Video 2024-02-03 > in production**

# Install Everything - 1 Time Run

In [ ]:
!yes | apt-get update --allow-releaseinfo-change
!yes | apt install libgoogle-perftools-dev
!mkdir -p /kaggle/temp
%cd /kaggle/temp
!git clone https://github.com/FurkanGozukara/SUPIR
%cd /kaggle/temp/SUPIR
%env PYTHONWARNINGS=ignore
%env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.9,max_split_size_mb:256
%cd /kaggle/temp/SUPIR
!pip install -r requirements.txt
!pip install torch==2.2.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --upgrade
!pip install xformers==0.0.24 --upgrade
!pip install bitsandbytes==0.43.0 --upgrade
!pip install huggingface_hub
!pip install flask
!pip install pyngrok
!yes | apt-get install ffmpeg



#  Download all models

In [ ]:
# analyze below wget commands to download different models as you wish
!mkdir -p /kaggle/temp/SUPIR/models
!mkdir -p /kaggle/temp/SUPIR/models/checkpoints
!wget "https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/resolve/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors"
!wget "https://civitai.com/api/download/models/360292?token=22c5b4cb1989d2c3ff29f222d2840884" -O "WildCardX_XL_Lightning.safetensors"   
%cd /kaggle/temp/SUPIR
!python download_models.py

### First you need to register and get a token. 
### I used gmail register >
### https://dashboard.ngrok.com/get-started/your-authtoken
### Skipped authenticator not necessary
### Here A Video Of How To Use NGROK : https://youtu.be/iBT6rhH0Fjs

In [ ]:

### Here A Video Of How To Use NGROK : https://youtu.be/iBT6rhH0Fjs
## first put your ngrok token to the below and then run this code
## it will give a link like this at below : https://2fc5-34-134-226-xxx.ngrok-free.app
## open it and then run web ui and once web ui started that link will start working
import os
import threading

from flask import Flask
from pyngrok import ngrok, conf

conf.get_default().auth_token = "REPLACE_WITH_YOUR_NGROK_TOKEN"

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)

public_url = ngrok.connect(7860).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 7860))

app.config["BASE_URL"] = public_url

@app.route("/")
def index():
    return "how to use Ngrok video : https://youtu.be/iBT6rhH0Fjs"

threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


In [ ]:
# run this cell to start the app then visit the above given link
# if you want to run gradio again rerun the above cell again and get a new link
# ignore the error if it says port in use when you run above cell
# the first generation will be slower always
%cd /kaggle/temp/SUPIR
#use this for FP16
#!python gradio_demo.py --use_tile_vae --loading_half_params --load_4bit_llava --dont_move_cpu --fast_load_sd
#use this for FP8
!python gradio_demo.py --fp8 --use_tile_vae --load_4bit_llava --dont_move_cpu --fast_load_sd


In [ ]:
#run this cell to generate a zip file of all generated images and then you can download
#them from the right panel as zip file
# Navigate to the parent directory of the outputs folder
!cd /kaggle/temp/SUPIR/

# Remove existing zip file if it exists
!rm -f ../images.zip

# Create a new zip file with only the outputs folder
!zip -r ../images.zip outputs

# Navigate back to the original working directory if needed
%cd /kaggle/temp/SUPIR
